In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from nn_analysis import models as md
from nn_analysis import utils
from nn_analysis.constants import MODEL_CONFIGS_PATH

model_configs = utils.load_config(MODEL_CONFIGS_PATH)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def exclude_bias_and_norm(p):
    return p.ndim == 1


# barlow P

In [15]:
epoch = 49
model_name = 'barlow_P'
model = md.get_custom_model(arch='equivariant_all_bn', extract_method='dpp', path='checkpoints/barlowtwins/equivariant_all_bn_v2/checkpoint.pth', device=device, state_dict_key='model')

Missing keys: []
Unexpected keys: []


In [21]:
state_dict = model.state_dict()
torch.save({'state_dict': state_dict}, '/mnt/smb/locker/issa-locker/users/hc3190/data/models/checkpoints/barlowtwins/equivariant_all_bn_v2_projector/0082.pth.tar')

In [5]:
model = md.get_custom_model(arch='resnet50', epoch=82, path='checkpoints/barlowtwins/equivariant_all_bn_v2', device=device)
model_projector = md.get_custom_model(arch='equivariant_all_bn', epoch=82, path='checkpoints/barlowtwins/equivariant_all_bn_v2_projector', device=device)

Missing keys: ['fc.weight', 'fc.bias']
Unexpected keys: []
Missing keys: []
Unexpected keys: []


In [15]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        assert torch.allclose(param, model_projector.state_dict()[f'backbone.{name}'])

# barlow control

In [18]:
model = md.get_custom_model(arch='barlow_twins', extract_method='dpp', path='checkpoints/barlowtwins/original_v2/checkpoint.pth', device=device, state_dict_key='model')

Missing keys: []
Unexpected keys: []


In [21]:
state_dict = model.state_dict()
torch.save({'state_dict': state_dict}, '/mnt/smb/locker/issa-locker/users/hc3190/data/models/checkpoints/barlowtwins/original_v2_projector/0054.pth.tar')

In [22]:
model = md.get_custom_model(arch='resnet50', epoch=54, path='checkpoints/barlowtwins/original_v2', device=device)
model_projector = md.get_custom_model(arch='barlow_twins', epoch=54, path='checkpoints/barlowtwins/original_v2_projector', device=device)

Missing keys: ['fc.weight', 'fc.bias']
Unexpected keys: []
Missing keys: []
Unexpected keys: []


In [23]:
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        assert torch.allclose(param, model_projector.state_dict()[f'backbone.{name}'])

In [26]:
model_projector = md.get_custom_model(arch='equivariant_all_bn', epoch=82, path='checkpoints/barlowtwins/equivariant_all_bn_v2_projector', device=device)
model_projector_base = md.get_custom_model(arch='barlow_twins', epoch=54, path='checkpoints/barlowtwins/original_v2_projector', device=device)

for key in list(model_projector.state_dict().keys()):
    if key not in list(model_projector_base.state_dict().keys()):
        print(key)
print("HI")
for key in list(model_projector_base.state_dict().keys()):
    if key not in list(model_projector.state_dict().keys()):
        print(key)

Missing keys: []
Unexpected keys: []
Missing keys: []
Unexpected keys: []
embedding
HI
